IDS 561 Group 24 Project Code

ML Model: Linear Regression

In [ ]:
#Connecting Google Drive to Colab
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
%cd /content/drive/MyDrive/IDS 561 Project/Google Colab Files

/content/drive/MyDrive/IDS 561 Project/Google Colab Files


In [ ]:
#Getting the Spark installer (check the path on spark.apache.org)
!wget -v https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

--2022-04-27 01:28:17--  https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 300971569 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop3.2.tgz.2’

spark-3.2.1-bin-had 100%[===================>] 287.03M  44.2MB/s    in 6.8s    

2022-04-27 01:28:24 (42.2 MB/s) - ‘spark-3.2.1-bin-hadoop3.2.tgz.2’ saved [300971569/300971569]



In [ ]:
#Check path
!ls

 data2019.csv				  IDS561ProjectRajaram.ipynb
 data2020.csv				  spark-3.2.1-bin-hadoop3.2
 data2021.csv				  spark-3.2.1-bin-hadoop3.2.tgz
'datafile_for_modelling (Old_Test).csv'   spark-3.2.1-bin-hadoop3.2.tgz.1
 Flight_Delay_Random_Forest_Model.ipynb   spark-3.2.1-bin-hadoop3.2.tgz.2
 flights_new_bal.csv			  Untitled0.ipynb
 full_df.parquet


In [ ]:
#Untar the Spark installer
!tar -xvf spark-3.2.1-bin-hadoop3.2.tgz

spark-3.2.1-bin-hadoop3.2/
spark-3.2.1-bin-hadoop3.2/LICENSE
spark-3.2.1-bin-hadoop3.2/NOTICE
spark-3.2.1-bin-hadoop3.2/R/
spark-3.2.1-bin-hadoop3.2/R/lib/
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/DESCRIPTION
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/INDEX
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/Meta/
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/Meta/Rd.rds
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/Meta/features.rds
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/Meta/hsearch.rds
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/Meta/links.rds
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/Meta/nsInfo.rds
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/Meta/package.rds
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/Meta/vignette.rds
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/NAMESPACE
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/R/
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/R/SparkR
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/R/SparkR.rdb
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/R/SparkR.rdx
spark-3.2.1-bin-hadoop3.2/R/lib/Sp

In [ ]:
#Installing findspark
!pip install -q findspark

In [ ]:
#Setting environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/drive/MyDrive/IDS 561 Project/Google Colab Files/spark-3.2.1-bin-hadoop3.2" 

In [ ]:
#Creating a local Spark session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import pandas as pd
import numpy as np

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler 
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.types import DoubleType
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder


sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

/content/drive/MyDrive/IDS 561 Project/Google Colab Files/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
datafile_for_modelling = spark.read.csv('datafile_for_modelling.csv', header = True, inferSchema = True)
datafile_for_modelling.show()

+---+----+-------+-----+------------+-----------+-----------------+---------------------+----------+--------+-----------------+---------------------+---------------------+----------------+----------------+-----------------+---------------+----------+---------------+-------------------+-------------------+----------------+--------------+---------------+-------------+--------+---------+-------------+---------+---------------+------------+--------+-------+--------+---------+-------------+---------+---------------+------------+-----------------+--------+----------------+-------------------+--------+-------+--------+--------------+-------------+-------------+---------+--------------+-------------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+
|_c0|YEAR|QUARTER|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|OP_CARRIER_AIRLINE_ID|OP_CARRIER|TAIL_NUM|ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|OR

Create Vector Assembler and train/test sets

In [ ]:
#Creating a Vector Assembler

select_features = ['DEP_DELAY', 'TAXI_OUT', 'CRS_ELAPSED_TIME', 'DISTANCE', 
                   'flight_weekday_1', 'flight_weekday_2', 'flight_weekday_3', 
                   'flight_weekday_4', 'flight_weekday_5', 'flight_weekday_6',
                   'flight_weekday_7']

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = select_features, outputCol = "features")

datafile_for_modelling_v2 = assembler.transform(datafile_for_modelling)
datafile_for_modelling_v2 = datafile_for_modelling_v2.select(['features', 'ARR_DELAY'])
datafile_for_modelling_v2.show()


+--------------------+---------+
|            features|ARR_DELAY|
+--------------------+---------+
|(11,[0,1,2,3,6],[...|    -27.0|
|(11,[0,1,2,3,7],[...|     -8.0|
|(11,[0,1,2,3,8],[...|    -14.0|
|(11,[1,2,3,9],[25...|      2.0|
|(11,[2,3,10],[96....|      0.0|
|(11,[2,3,4],[96.0...|      0.0|
|(11,[2,3,5],[96.0...|      0.0|
|(11,[0,1,2,3,6],[...|     13.0|
|(11,[0,1,2,3,7],[...|     13.0|
|(11,[0,1,2,3,8],[...|     -2.0|
|(11,[0,1,2,3,9],[...|    -19.0|
|(11,[0,1,2,3,10],...|     18.0|
|(11,[0,1,2,3,4],[...|      6.0|
|(11,[0,1,2,3,5],[...|     -1.0|
|(11,[0,1,2,3,6],[...|     13.0|
|(11,[0,1,2,3,7],[...|    -16.0|
|(11,[1,2,3,5],[13...|    -17.0|
|(11,[0,1,2,3,6],[...|    -17.0|
|(11,[0,1,2,3,7],[...|    -22.0|
|(11,[0,1,2,3,8],[...|    -11.0|
+--------------------+---------+
only showing top 20 rows



In [ ]:
train, test = datafile_for_modelling_v2.randomSplit([0.7, 0.3])

Linear Regression w/o Regularization

In [ ]:
from pyspark.ml.regression import LinearRegression

#Creating the model
linear_reg_1 = LinearRegression(featuresCol = "features", labelCol = "ARR_DELAY")
linear_reg_model_1 = linear_reg_1.fit(train)

In [ ]:
trainingSummary_v1 = linear_reg_model_1.summary

print("RMSE: %f" % trainingSummary_v1.rootMeanSquaredError)
print("r2: %f" % trainingSummary_v1.r2)

RMSE: 11.769504
r2: 0.937185


Evaluate performance on test dataset

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

#Transforming test set
predictions_v1 = linear_reg_model_1.transform(test)
eval_reg = RegressionEvaluator(labelCol="ARR_DELAY", metricName="r2")

In [ ]:
print("R2 on test data = %g" % eval_reg.evaluate(predictions_v1))
test_result_v1 = linear_reg_model_1.evaluate(test)
print("RMSE on test data = %g" % test_result_v1.rootMeanSquaredError)

R2 on test data = 0.937877
RMSE on test data = 11.7292


In [ ]:
print("Best Param (regParam): %g" % linear_reg_model_1._java_obj.getRegParam())

print("Best Param (MaxIter):  %g" %linear_reg_model_1._java_obj.getMaxIter())

print("Best Param (elasticNetParam):  %g" % linear_reg_model_1._java_obj.getElasticNetParam())

Best Param (regParam): 0
Best Param (MaxIter):  100
Best Param (elasticNetParam):  0


Linear Regression w/ Regularization

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression

linear_reg_2 = LinearRegression(labelCol = 'ARR_DELAY', featuresCol = 'features')
pipeline = Pipeline(stages=[linear_reg_2])

paramGrid = ParamGridBuilder()\
    .addGrid(linear_reg_2.regParam, [0]) \
    .addGrid(linear_reg_2.elasticNetParam, [0.0])\
    .addGrid(linear_reg_2.maxIter, [100])\
    .build()

cross_val = CrossValidator(estimator = pipeline, estimatorParamMaps = paramGrid,
                           evaluator = RegressionEvaluator(labelCol = "ARR_DELAY"), numFolds = 3)

linear_reg_model_2 = cross_val.fit(train)

In [ ]:
#Getting the best model
bestModel = linear_reg_model_2.bestModel

In [ ]:
#Getting the summary scores on the training set
trainingSummary_v2 = linear_reg_model_2.bestModel.stages[-1].summary

print("RMSE: %f" % trainingSummary_v2.rootMeanSquaredError)
print("r2: %f" % trainingSummary_v2.r2)

RMSE: 11.769504
r2: 0.937185


In [ ]:
#Predicting on the test set
predictions_v2 = linear_reg_model_2.transform(test)
eval_reg_2 = RegressionEvaluator(labelCol="ARR_DELAY", metricName="r2")

In [ ]:
#Pringing out results
print("R Squared (R2) on test data = %g" % eval_reg_2.evaluate(predictions_v2))
test_result_v2 = linear_reg_model_1.bestModel.stages[-1].evaluate(test)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result_v2.rootMeanSquaredError)

R Squared (R2) on test data = 0.937877
Root Mean Squared Error (RMSE) on test data = 11.7292


In [ ]:
#Printing the best parameters
print("Best Param (regParam): %g" % bestModel.stages[-1]._java_obj.getRegParam())

print("Best Param (MaxIter):  %g" % bestModel.stages[-1]._java_obj.getMaxIter())

print("Best Param (elasticNetParam):  %g" % bestModel.stages[-1]._java_obj.getElasticNetParam())

Best Param (regParam): 0
Best Param (MaxIter):  100
Best Param (elasticNetParam):  0
